In [1]:
# dependencies
import ast          # new library: abstract syntax trees!
import pandas as pd
import pymongo
import re           # new library: regex!

In [2]:
# connect to mongo database "DataMate"
conn = 'mongodb://localhost:27017'
cli = pymongo.MongoClient(conn)
db = cli.DataMate

In [3]:
# read JSON from file. was "test_output.txt", is now "pof_output.txt"
# encoding was "utf-8", is now "cp1252"
strPath = "../scraping/pof_output.txt"
with open(strPath, 'r', encoding = 'cp1252') as file:
    strFile = file.read()
    dictProfile = ast.literal_eval(strFile)

In [4]:
# write Profiles to DataMate db table "Profile"
db.Profile.insert_many([dictProfile])

In [5]:
# the CountWords function breaks "hi...let's" into "hi" and "let's"
def CountWords(strChunk, dict):
    lst = re.split(r"[^a-zA-Z0-9'\-]+", strChunk) # split on all chars except letters, numbers, and apostrophes
    for strWord in lst:
        if len(strWord) > 0:
            dict = CountWord(strWord, dict)
    return dict

In [6]:
# the CountWord function logs word counts in dictAboutMe
def CountWord(strWord, dict):
    if strWord in dict:    # have we already logged this word?
        dict[strWord] += 1 # if yes: increment word's count
    else:
        dict[strWord] = 1  # if no: add row to dict
    return dict

In [7]:
# instantiate word frequency dictionaries
dictAboutMe = {}
dictInterest = {}

In [8]:
# populate "About Me" dictionary
for strKey, strVal in dictProfile.items():
    if(strKey != '_id'):
        for strChunk in strVal['about_me_split']:
            dictAboutMe = CountWords(strChunk.lower(), dictAboutMe)

In [9]:
# populate "Interest" dictionary
for strKey, strVal in dictProfile.items():
    if(strKey != '_id'):
        for strChunk in strVal['interests']:
            dictInterest = CountWord(strChunk.lower(), dictInterest)

In [10]:
# convert dictionaries to sorted dataframes
dfAboutMe = pd.DataFrame.from_dict(dictAboutMe, orient='index', columns=['count'])
dfAboutMe = dfAboutMe.sort_values(['count'], ascending=False)
dfInterest = pd.DataFrame.from_dict(dictInterest, orient='index', columns=['count'])
dfInterest = dfInterest.sort_values(['count'], ascending=False)

In [11]:
# build list of words to exclude
lstExclude = ['a', 'am', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'e', \
              'for', 'if', 'in', 'is', 'it', 'n', 'not', 'o', 'of', 'oh', \
              'on', 'or', 'out', 'so', 'that', 'the', 'to', 'was', 'with']

In [12]:
# remove boring words from dataframes
dfAboutMe = dfAboutMe.drop(lstExclude)
dfInterest = dfInterest.drop(lstExclude)

KeyError: "['a' 'am' 'an' 'and' 'are' 'as' 'at' 'be' 'but' 'for' 'if' 'in' 'is' 'it'\n 'not' 'of' 'oh' 'on' 'or' 'out' 'so' 'that' 'the' 'to' 'was' 'with'] not found in axis"

In [ ]:
# export dataframes to CSVs
dfAboutMe.to_csv('AboutMe.csv', index=True, header=True)
dfInterest.to_csv('Interest.csv', index=True, header=True)